# Praktikum 2 - Generator Teks dengan RNN

## Setup
Import TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


Load Data

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


## Olah Teks
Vectorize Teks

In [8]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
print(chars)

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


In [10]:
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

In [12]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [13]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [14]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [16]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [17]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

## Prediksi
Membuat Trianing Set dan Target

In [19]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [20]:
seq_length = 100

In [21]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [22]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [27]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list("Tensorflow"))

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


## Membuat Batch Training

In [28]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Buat Model

In [29]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [30]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [31]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Uji Model

In [32]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [33]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [36]:
sampled_indices

array([52, 46, 57, 20,  0, 25,  4, 39, 11, 16,  8, 52, 11, 58, 59, 26, 56,
       54, 39, 16, 39, 57, 64, 23, 48, 60, 62, 12, 37, 46, 56,  0, 26, 58,
       44, 12, 44,  5, 19, 23, 60,  8, 10, 35, 64, 63, 37, 62, 42, 19, 10,
       22, 55, 33, 56, 49,  6, 32, 33, 34, 22, 26, 13, 54, 13, 57, 31,  1,
       20, 26, 26, 30, 57, 42, 25, 63, 30, 30, 64,  3, 47, 65, 21, 62, 11,
        2, 20, 31, 38, 60, 41, 52,  6, 60,  9, 23, 18, 60, 60,  9])

In [37]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b" give out a\ncommission for more heads: if this law hold in\nVienna ten year, I'll rent the fairest ho"

Next Char Predictions:
 b"mgrG[UNK]L$Z:C-m:stMqoZCZryJiuw;Xgq[UNK]Mse;e&FJu-3VyxXwcF3IpTqj'STUIM?o?rR\nGMMQrcLxQQy!hzHw: GRYubm'u.JEuu."


## Train Model
Tambahan optimizer dan fungsi loss

In [39]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [40]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189408, shape=(), dtype=float32)


In [41]:
tf.exp(example_batch_mean_loss).numpy()

65.9837

In [43]:
model.compile(optimizer='adam', loss=loss)

Konfigurasi Checkpoints

In [44]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Lakukan Proses Training

In [47]:
EPOCHS = 5

history = model.fit(dataset, epochs=EPOCHS, steps_per_epoch=10, callbacks=[checkpoint_callback])

Epoch 1/5
10/10 [==============================] - 59s 6s/step - loss: 4.2761
Epoch 2/5
10/10 [==============================] - 63s 7s/step - loss: 3.7716
Epoch 3/5
10/10 [==============================] - 72s 7s/step - loss: 3.2611
Epoch 4/5
10/10 [==============================] - 52s 5s/step - loss: 3.0690
Epoch 5/5
10/10 [==============================] - 53s 5s/step - loss: 2.8871


## Generate Teks

In [48]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [49]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [50]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
; tfs lby atepteur iehahs t but hon.
XK lsI qud I
dtnga col f hlikenug bnerg  yun ., due
means, g wt;s hugo
Hthia nipajnad hordd s ior aon s,, hr
alet b ae oo a,
Ts.
.ddbErskreTwfhhyt.
Q
&TPSmWlhn knalni;e
COGx:;
een poin fadpern ateqmoiI,
EYGtvowa
aro newvfe are, soeur, art ans yhforele ciaod ghalH
r phthlain abpuo tho t;kberatkisd wo thd,la'fodbfe eiwntin:t mI'the wpr eirt inmen,sd

cl?
a
D hete


x'ohnsem, w entsdd chit yuiIind set ,n m lnu we hoh,d rt
Thocedvy we y phe pe bolpfe soadk fvwie icl, gopwoouc thank y mnMiticiuk at atcsrrs an erhe tn toddemaais cn
I :cm EcbyasouYU
KLr
Ve hmmond br mipo the.
tcwedsu lutfd mh. mii, po aagin her sar hhun this haw.
Senhurk ieas ss anheawfy vwhe'tfs the id hay amt wyegr,



?Zjvper

 Hipggrane ea'wliun thio la
oe tI cten osdhes hbe: wlon m n tho wto
oeain aa i h ua,nr
A

,cdJRva hoohdy
.EM
!qwxy olrae se t ldde
t bthoari hud in thd bdavaded ian dafh ute, wrfsooeusiut ad eo ms piu

Avrl oud y te avoontm n whe tithhingleanno efo iee s us

In [51]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:ls athi tane\n\nInca.\nZdBwiltched sod ahis iop mol ee nny ed t mea yhee it y\nchydo\nf\nFsicthir al'r e or thm hot thesener tsed\n\nN,Tala:\nyuure t ll\nI olthfsg sd cmeoy d, arnn\n.k\nmyy f o ol aoarirldsysfegw.\nS\nq..H3I lpa.\n\nNYNNLR?&e\nHd!; fn auul oh'wlk bs si, firhy woas eh, safm d borntsilhyimu wo me t thed\nIAbhgI m y fhume soredol iad yn\nfncdiTmh,d roel\nV\nxwcWessa't gouaf:k ghe snr at th te e ge bus pe \naalguoe pef lAgpy wyr as e end tis aly wy be, te w in isnt wa\ntr ot ilo maps! tgiile cinou agaialthos  ato, mhse they oul irrmse seredr lo lo bpeh\n cclirev wan Hao hot mrhe welt secegectouko'na is wd parl y, t nurh llis\nWotrel ao oi ny and\nF Ihoe wiiny warllk. p,eu\nXlf'lI uynm i\n' swy  tatdn\nS:\n\nEyeCo;z: wr aoit ons rsm\nxisn hes, s ah,\nDU\nTGb. Shy pdehho use s fy eiwd uom tce ar i\niWnggcew vre wi\nD ooy t;cifn.\nuthyhe, oo;\nYK\nIYvD?DM'a all t trehut hilh oan or la roene pme bane nl por re,\n\ny kynpntut, usen,\nSAkGurn loie arsoad a, t

## Ekspor Model Generator

In [52]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [61]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:Hd, ttcyo arn gbi;mashe ome wamele
Whsoyayis Sa'calite toikor, e m i a il as tos,, shh thppr basn, v
